# Metrics/Cost Functions: MetricBase, Infidelity, TraceDistance, Metrics

## Overview [Public API]

We want to consider these metrics/cost functions from an optimization/inverse design perspective. **These cost functions (aka fitness functions) quantify the quality of produced circuits.**

Inverse design is a design paradigm where we first pick a cost function, and then search the design space in the goal of minimizing/maximizing said function. In our algorithms, we only consider minimization (note however, that it is trivial to change a cost function which should be maximized into a minimizable cost function: you can simply add a negative sign). Most optimization algorithms work best when solutions which are nearby to optimal solutions have a low cost, and solutions which are further from optimal solutions have a high cost. 

Different requirements on quantum systems might lead to different definitions of cost functions. Additionally, some cost functions may be easier to optimize than others. Finally, we may want to evaluate metrics which we are not actively optimizing to check properties of our circuits. Thus, we support a number of cost functions/metrics which may perform differently under optimization.

We will consider the following:
1. General features of metrics/cost functions
2. `Infidelity`, `TraceDistance`, and `CircuitDepth`
3. Joint metrics (trading off between metric classes)

## Metric object requirements: `MetricBase`

All metric objects inherit from `MetricBase`

### 1. Initialization

All metric can be initialized with a `log_steps` argument. If unspecifies, `log_steps` defaults to one.

Each `MetricBase` type object will log metric values upon evaluation. They will log every `log_step` value. This can be useful to tracking metrics over the run of certain solvers (particularly those which are not population based); however, the logging information collected through the solver is likely more useful.

Sometimes, metrics will require additional initialization information (e.g. a target state). This is not always required.

### 2. Evaluation

Each metric object must have an evaluate function, `evaluate(state, circuit)`; that is, we consider the quality of a solution circuit to be fully characterized by its components and the state it produces.

Metrics may only depend on the produced state(e.g. `Infidelity`,  `TraceDistance`), or only depend on the circuit (e.g. `CircuitDepth`). However, we require both arguments such that the solver can be agnostic to the metric type (i.e. so that it can pass both arguments without having to check the type of metric which it is running).

`evaluate` must return a scalar (later, a vector may be acceptable for pareto-front optimization. This is not the case at the moment)

# `Infidelity`, `TraceDistance`, `CircuitDepth`

These are currently implemented metrics in our system.

`Infidelity` and `TraceDistance` are measures of closeness between two quantum states (where smaller values indicate closer quantum states). Thus, we can make our optimizable cost functions the infidelity/trace distance between our produced state, and a target state.

**NOTE:** both of these metrics require the two quantum states being compared to have the same size (i.e. same number of qubits). Otherwise, and error will be raised.

In [1]:
""" Evaluating metrics """

from benchmarks.circuits import ghz4_state_circuit, linear_cluster_4qubit_circuit
import src.metrics as met
from src.backends.density_matrix.state import DensityMatrix

# Let's look at a ghz4 target state
ghz4_circuit, ghz4_target = ghz4_state_circuit()
ghz4_target = ghz4_target['dm']  # this gives us a numpy array. A change coming in will force this to be a QuantumState

# Initialize metrics
infidelity = met.Infidelity(ghz4_target)
trace_dist = met.TraceDistance(ghz4_target)
circ_depth = met.CircuitDepth()

# Let's look at optimal results
print(f'Cost functions results on perfect state/circuit:')
print(f'Infidelity: {infidelity.evaluate(ghz4_target, ghz4_circuit)}')
print(f'Trace distance: {trace_dist.evaluate(ghz4_target, ghz4_circuit)}')
print(f'Circuit depth: {circ_depth.evaluate(ghz4_target, ghz4_circuit)}')

# Let's look at results, where the circuit produces the wrong state (for example, the linear cluster state)
linear4_circuit, linear4_target = linear_cluster_4qubit_circuit()
linear4_target = linear4_target['dm']

print(f'\nCost functions results on an incorrect state/circuit:')
print(f'Infidelity: {infidelity.evaluate(linear4_target, ghz4_circuit)}')
print(f'Trace distance: {trace_dist.evaluate(linear4_target, ghz4_circuit)}')
print(f'Circuit depth: {circ_depth.evaluate(linear4_target, ghz4_circuit)}')

# We can look at the logged values (recalling that by default, log_steps == 1, and we log every function result)
print(f'\nInfidelity log: {infidelity.log}')
print(f'\n Trace Distance log: {trace_dist.log}')
print(f'\n Circuit depth log: {circ_depth.log}')

Cost functions results on perfect state/circuit:
Infidelity: 4.440892098500626e-16
Trace distance: 0.0
Circuit depth: 8

Cost functions results on an incorrect state/circuit:
Infidelity: 1.0
Trace distance: 1.0
Circuit depth: 8

Infidelity log: [4.440892098500626e-16, 1.0]

 Trace Distance log: [0.0, 1.0]

 Circuit depth log: [8, 8]


### CircuitDepth metric: normalization

While the `Infidelity` and `TraceDistance` metrics have an obvious normalization, this is not the case for `CircuitDepth`. In the example above, we did not normalize circuit depth at all. However, we also allow a `depth_penalty` function to be defined.

In [2]:
""" CircuitDepth metric: normalization """

# Suppose that we don't mind circuits of depth ~< 16, but we want to punish circuit growth more harshly afterwards

circ_depth_quadratic = met.CircuitDepth(depth_penalty=lambda x: (x / 16) ** 2)
print(f'Circuit depth penalty: {circ_depth_quadratic.evaluate(linear4_target, ghz4_circuit)}')

Circuit depth penalty: 0.25


### Function implementation [Implementer Info]

`Infidelity` and `TraceDistance` are currently only implemented in the density matrix representation (and the state input must reflect this--currently the state input is a numpy array, but shortly a change will come in and it will be a `QuantumState` object. Nevertheless, the `QuantumState` object must have a density matrix representation). An upcoming change will add the option to run `Infidelity` in stabilizer formalism.

They are implemented from helper functions in `src/backends/density_matrix/functions.py`.

`CircuitDepth` is implemented from a `depth` attribute in the circuit class.

## Joint metrics

It can be useful to consider multiple metrics at once in our cost function. For example, we may decide that we care both about fidelity (how good the produced state is) and circuit depth (a measure of how expensive it would be to produce this state using the designed circuit).

For this purpose, we can use the `Metrics` class.

In [3]:
""" Combo metric, default weighting """

combo_metric = met.Metrics([infidelity, trace_dist, circ_depth_quadratic])
print(f'Combined metric on correct state/circuit: {combo_metric.evaluate(ghz4_target, ghz4_circuit)}')
print(f'Combined metric on incorrect state/circuit: {combo_metric.evaluate(linear4_target, ghz4_circuit)}')

Combined metric on correct state/circuit: 0.25000000000000044
Combined metric on incorrect state/circuit: 2.25


In [4]:
""" Combo metric, linear combination weighting """
combo_metric = met.Metrics([infidelity, trace_dist, circ_depth_quadratic], metric_weight=[0.4, 0.4, 0.2])
print(f'Combined metric on correct state/circuit: {combo_metric.evaluate(ghz4_target, ghz4_circuit)}')
print(f'Combined metric on incorrect state/circuit: {combo_metric.evaluate(linear4_target, ghz4_circuit)}')

Combined metric on correct state/circuit: 0.05000000000000018
Combined metric on incorrect state/circuit: 0.8500000000000001


In [7]:
""" Combo metric, arbitrary weighting / metric function """

# first, let's clear the metric_list logs
infidelity.log = []
trace_dist.log = []
circ_depth_quadratic = []

metric_list = [infidelity, trace_dist, circ_depth_quadratic]
def metric_weight_custom(state, circuit):
    metrics_eval = 0.4 * metric_list[0].evaluate(state, circuit) + 0.4 * metric_list[1].evaluate(state, circuit) + \
                   0.2 * metric_list[2].evaluate(state, circuit) ** 2
    return metrics_eval

combo_metric = met.Metrics(metric_list, metric_weight=metric_weight_custom)
print(f'Combined metric on correct state/circuit: {combo_metric.evaluate(ghz4_target, ghz4_circuit)}')
print(f'Combined metric on incorrect state/circuit: {combo_metric.evaluate(linear4_target, ghz4_circuit)}')

TypeError: unhashable type: 'list'

In [6]:
""" Finally, we can still access the log of a particular metric, as well as the logs of each constituent function"""

print(f'score log:')
print(combo_metric.log)

print(f'score log for each constituent function:')
print(combo_metric.per_metric_log)

score log:
[0.012500000000000178, 0.8125]
score log for each constituent function:
{'Infidelity': [4.440892098500626e-16, 1.0, 4.440892098500626e-16, 1.0, 4.440892098500626e-16, 1.0, 4.440892098500626e-16, 1.0], 'TraceDistance': [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0], 'CircuitDepth': [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]}
